In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

In [ ]:
from utils import prepare_data, cost_func, evaluate_model

In [ ]:
from custom_estimator import IdentifyRepeats, TimeDiff

In [ ]:
X_train, X_test, y_train, y_test = prepare_data()

In [ ]:
pip_cat = Pipeline([
    ("encoder", OneHotEncoder()),
])

selector = ColumnTransformer([
    ("numerical", "passthrough", ["age", "purchase_value"]),
    ("categorical", pip_cat, ["sex", "browser", "source"]),
    ("repated_devices", IdentifyRepeats(), "device_id"),
    ("time_diff", TimeDiff(), ["signup_time", "purchase_time"])
])

model = Pipeline([
    ("selector", selector),
    ("classifier", DecisionTreeClassifier(max_depth=5))
])

In [ ]:
param_grid = {"classifier__max_depth": range(1,20), "classifier__class_weight": [None, "balanced"]}
grid_search = GridSearchCV(model, param_grid, cv=5, verbose=1, scoring=cost_func)

In [ ]:
import custom_estimator
import importlib
importlib.reload(custom_estimator)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
evaluate_model(grid_search, X_train, y_train, "Training")

In [ ]:
cost_func(grid_search, X_train, y_train)/ X_train.shape[0]

In [ ]:
evaluate_model(grid_search, X_test, y_test, "Testing")

In [ ]:
cost_func(grid_search, X_test, y_test)/ X_test.shape[0]

In [ ]:
grid_search.best_params_

##The effect of Threshold

In [ ]:
import numpy as np
thresholds = np.linspace(0, 1, 100)

In [ ]:
def predict_labels(model, X, threshold=0.5):
    return (model.predict_proba(X)[:, 1] > threshold).astype(int)

In [ ]:
predict_labels(grid_search, X_train)

In [ ]:
def cost_func_w_threshold(model, X, y_true, threshold=0.5):
    """
    Return cost of model based upon FP and FN
    Cost = 7 * FP + purchase value of each FN
    """
    y_pred = (model.predict_proba(X)[:, 1] > threshold).astype(int)
    FP = (y_pred & ~y_true).sum()

    # FN: we say it's NOT fraudulent (y=0) AND it's truely Fraudulent (y_true = 1)
    FN = X.loc[(~y_pred & y_true).index, "purchase_value"].sum()

    return -(7 * FP + FN)

In [ ]:
cost_func_w_threshold(grid_search, X_train, y_train) / X_train.shape[0]

In [ ]:
#probably i should tweak the way i find the best model
#for a given set of hyper params, there is an optimal threshold
#And by optimal, i mean a value for threshold that will have the lowest cost func value

In [ ]:
model.set_params(classifier__max_depth=2)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.predict_proba(X_train)[:,1]

In [ ]:
cost_func_w_threshold(model, X_train, y_train, threshold=0.4) / X_train.shape[0]

In [ ]:
costs = np.array([
                  cost_func_w_threshold(model,X_train, y_train, threshold=threshold) for threshold in thresholds
                  ]) / X_train.shape[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(thresholds, costs, '--r')

##Trying Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pip_cat = Pipeline([
    ("encoder", OneHotEncoder()),
])

selector = ColumnTransformer([
    ("numerical", "passthrough", ["age", "purchase_value"]),
    ("categorical", pip_cat, ["sex", "browser", "source"]),
    ("repated_devices", IdentifyRepeats(), "device_id"),
    ("time_diff", TimeDiff(), ["signup_time", "purchase_time"])
])

model = Pipeline([
    ("selector", selector),
    ("classifier", RandomForestClassifier())
])

In [ ]:
param_grid = {"classifier__max_depth": range(1,20), "classifier__class_weight": [None, "balanced"]}
grid_search = GridSearchCV(model, param_grid, cv=5, verbose=3, scoring=cost_func)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
#for the business as usual approach
#sum the purchase value of all fraud

In [ ]:
X_train.assign(label=y_train).query("label == 1")["purchase_value"].sum() / X_train[0]

In [ ]:
X_test.assign(label=y_test).query("label == 1")["purchase_value"].sum() / X_test[0]